## Mount Google Drive

In [ ]:
from google.colab import drive

# Make sure to unmount drive at mount point
drive.flush_and_unmount()
drive.mount('/content/drive')

# Research Start

In [1]:
from datetime import datetime
from scipy import stats  
import os
import pytz
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np  

%matplotlib inline

### Creating IDX Equities Time Series

In [7]:
idx_combined = pd.read_csv('/home/nikki/documents/amibroker/idx_exported_combined.csv', parse_dates=[1])

In [2]:
# Location of the IDX Stocks OHLC csv files
# csv_loc = '/content/drive/Shared drives/algo-clenow/idx_exported_csv'
# csv_loc = '/Users/nikki/Documents/idx_exported_csv'
csv_loc = '/home/nikki/documents/amibroker/exported-csv/'

# Backup if the first one doesn't work
# csv_loc = '/content/drive/Shared drives/algo-clenow/idx_exported_csv_gama'

tickers = os.listdir(csv_loc)

In [3]:
"""
Create a dictionary where the key is the ticker
and the value is a pandas dataframe of the OHLC time series
"""
data_idx = {}
for ticker in tickers:
    data_idx[ticker[:-4]] = pd.read_csv(f'{csv_loc}/{ticker}',
                                        index_col='date',
                                        parse_dates=['date'])

In [4]:
data_idx['ASII'].tail()

,open,high,low,close,volume
date,,,,,
2020-08-31,5325.0,5350.0,5100.0,5100.0,42240600
2020-09-01,5175.0,5250.0,5100.0,5250.0,20881000
2020-09-02,5300.0,5350.0,5200.0,5300.0,19074500
2020-09-03,5350.0,5350.0,5125.0,5175.0,38323300
2020-09-04,5100.0,5150.0,5025.0,5050.0,47966200


### Stocks on the Move


In [5]:
def momentum_score(ts):
    """
    Input:  Price time series.
    Output: Annualized exponential regression slope, 
            multiplied by the R2
    """
    # Make a list of consecutive numbers
    x = np.arange(len(ts)) 
    # Get logs
    log_ts = np.log(ts) 
    # Calculate regression values
    slope, intercept, r_value, p_value, std_err = stats.linregress(x, log_ts)
    # Annualize percent
    annualized_slope = (np.power(np.exp(slope), 252) - 1) * 100
    #Adjust for fitness
    score = annualized_slope * (r_value ** 2)
    return score

In [6]:
def volatility(ts, period=24):
    """
    Input:  Price time series, Look back period
    Output: Standard deviation of the percent change
    """
    return ts.pct_change().rolling(period).std().iloc[-1]

In [7]:
# How many (series) candles back for momentum calculation?
momentum_window = 96

# Create an empty DataFrame to store score
momentum_table = pd.DataFrame(columns=['ticker', 'score', 'vola', 'inv_vola'])

# How many (series) candles+1 back for std dev calculation?
vola_window = 24

# How many (series) candles back for EWMA calculation?
ewma_period = 100

# Loop the dictionary and calculate the momentum_score, then append it to pandas
for ticker, timeseries in data_idx.items():
    # Need the stocks to exist at least 3 years prior (756 trading days)
    if len(timeseries) < 756:
        continue
        
    # If there's a 0 volume in the stocks, drop it 
    if 0 in timeseries['volume'].iloc[-momentum_window:].tolist():
        continue

    momentum_series = timeseries['close'].iloc[-momentum_window:]
    score = momentum_score(momentum_series)
    vola_series = timeseries['close']
    vola = volatility(vola_series, vola_window)
    avg_volume = timeseries['volume'].rolling(vola_window).mean().iloc[-1] / 100
    ewma = timeseries['close'].ewm(span=ewma_period).mean().iloc[-1]
    
    if avg_volume < 500:
        continue

    momentum_table = momentum_table.append({'ticker': ticker,
                                            'score': score,
                                            'vola': vola,
                                            'avg_vol': avg_volume,
                                            'ewma': ewma},
                                           ignore_index=True)
    
    momentum_table['inv_vola'] = 1 / momentum_table['vola']

In [8]:
momentum_table.sort_values('score', ascending=False)[:100].to_clipboard()

In [9]:
momentum_table.sort_values('score', ascending=False)[:30]

,ticker,score,vola,inv_vola,avg_vol,ewma
0,WIIM,4162.671908,0.065195,15.338557,8.820249e+05,237.506775
19,AGRO,3288.260306,0.052813,18.934838,1.642194e+06,211.041842
118,CENT,2487.975376,0.058725,17.028537,2.514775e+05,98.917807
15,MARK,2356.605924,0.027164,36.813279,8.267250e+04,671.911297
47,KRAS,1540.047676,0.026255,38.088650,1.215897e+06,310.212638
46,DOID,1025.617685,0.043406,23.038073,1.191699e+06,215.935937
83,NIKL,904.989937,0.040313,24.806167,1.051476e+05,665.527858
161,SMDR,799.552955,0.044527,22.458320,1.466805e+05,220.811473
92,IMJS,719.663460,0.070410,14.202521,3.890621e+05,237.058121
201,SMBR,617.603444,0.052460,19.062144,5.584387e+05,421.790696


In [10]:
momentum_table.sort_values('score', ascending=False)[30:50]

,ticker,score,vola,inv_vola,avg_vol
21,PTRO,225.927561,0.018699,53.478241,14820.300
109,BNII,211.639909,0.039131,25.555319,195759.398
165,INCO,205.520434,0.019923,50.193673,144604.250
52,MDKA,205.463693,0.031363,31.884485,1375980.348
89,TKIM,197.257865,0.021971,45.515323,132050.600
217,ADES,194.964266,0.028279,35.361699,2382.450
67,BMRI,186.295963,0.021418,46.688772,506511.100
54,LSIP,179.183145,0.018393,54.368642,453676.756
201,CPIN,174.288533,0.012622,79.224323,89611.150
35,ERAA,171.742505,0.025366,39.422616,223138.950


In [10]:
momentum_table.sort_values('score', ascending=False)[50:100]

,ticker,score,vola,inv_vola,avg_vol,ewma
1,AKRA,131.638805,0.020098,49.757163,2.230546e+05,2675.290229
22,IPOL,122.141408,0.073553,13.595707,8.200000e+03,74.833082
170,CEKA,118.632404,0.028442,35.158659,2.203958e+03,1815.320273
189,DKFT,117.621931,0.032464,30.803798,1.150092e+05,113.791470
193,LPCK,111.867498,0.035906,27.850581,2.029225e+04,811.556362
30,DSNG,107.900255,0.032604,30.670830,8.335054e+04,381.026508
134,MBSS,107.281962,0.020363,49.109648,1.634833e+04,374.557099
76,SMGR,105.445006,0.022895,43.677280,6.046042e+04,9625.164202
52,SDPC,105.347025,0.042359,23.607586,4.698958e+03,108.620447
162,BTPN,103.324598,0.019279,51.869512,1.712917e+03,2255.569081
